In [1]:
%pip install -i https://test.pypi.org/simple/ enet-seifeddine-dridi

Looking in indexes: https://test.pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.


Setup the project configuration

In [2]:
from time import time

import torch
from tqdm import tqdm
from lanenet.enet.config import EnetConfig
from lanenet.enet.model_utils import load_model, eval_model, compute_loss

In [3]:
#pretrained_model_path = 'pretrained_model/enet_model_894.pt'
pretrained_model_path = None
config = EnetConfig(pretrained_model_path=pretrained_model_path, train_full_model=False, max_epoch=1000, dataset_root_folder='../datasets/cityscapes/data_unzipped')
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
torch.multiprocessing.set_start_method('spawn')
torch.set_flush_denormal(True)
model, optimizer, train_dataset, test_dataset = load_model(config, device)
train_dataset_iter = iter(train_dataset)
test_dataset_iter = iter(test_dataset)

last_checkpoint_saving_time = time()
saving_period = 2 * 60  # 2 minutes
best_eval_loss = float('inf')

In [4]:
bar_format = '{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}, ''{rate_noinv_fmt}{postfix}]'
progress_bar = tqdm(range(0, config.max_epoch), bar_format=bar_format)
for epoch in progress_bar:
    try:
        in_tensor, target = next(train_dataset_iter)
    except StopIteration:
        # Iterator is exhausted
        train_dataset_iter = iter(train_dataset)
        in_tensor, target = next(train_dataset_iter)
    in_tensor = in_tensor.to(device)
    target = target.to(device)
    logits = model(in_tensor)
    loss = compute_loss(logits, target, config.custom_weight_scaling_const, config.scaling_props_range)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    progress_bar.set_description(f"Epoch [{epoch}/{config.max_epoch}]")
    progress_bar.set_postfix(loss=loss.item())
    if time() - last_checkpoint_saving_time >= saving_period:
        last_checkpoint_saving_time = time()
        try:
            eval_loss = eval_model(model, test_dataset_iter, config.custom_weight_scaling_const,
                                   config.scaling_props_range, device)
        except StopIteration:
            # Iterator is exhausted
            test_dataset_iter = iter(test_dataset)
            eval_loss = eval_model(model, test_dataset_iter, config.custom_weight_scaling_const,
                                   config.scaling_props_range, device)
        if eval_loss < best_eval_loss:
            config.save_checkpoint(model, optimizer, loss, epoch, '../')
            best_eval_loss = eval_loss

Epoch [71/1000]:   7%|▋         | 72/1000 [01:45<42:47,  0.36it/s, loss=1.87]


Evaluation Loss=1.9080379128456115
Epoch=72/1000, Loss=1.8723981380462646
Saving training checkpoint as ../pretrained_model/enet_model_encoder_71.pt at 2024-04-27 09:29:09.462121


Epoch [150/1000]:  15%|█▌        | 151/1000 [03:46<40:52,  0.35it/s, loss=2.08]


Evaluation Loss=1.9842722415924072


Epoch [226/1000]:  23%|██▎       | 227/1000 [05:47<39:08,  0.33it/s, loss=1.66]

...

Epoch [865/1000]:  87%|████████▋ | 866/1000 [24:15<06:27,  0.35it/s, loss=1.52]


Evaluation Loss=1.7057139873504639
Epoch=866/1000, Loss=1.521970510482788
Saving training checkpoint as ../pretrained_model/enet_model_encoder_865.pt at 2024-04-27 09:51:39.467080


Epoch [923/1000]:  92%|█████████▏| 924/1000 [26:16<03:34,  0.35it/s, loss=1.73]


Evaluation Loss=2.4688814520835876


Epoch [999/1000]: 100%|██████████| 1000/1000 [28:07<00:00,  0.59it/s, loss=1.46]
